### Importações e dicionários

In [122]:
import pandas as pd
import numpy as np
import os, sys
import datetime as dt
import psycopg2
import re
from glob import glob
pd.set_option('display.float_format', '{:,.2f}'.format)

In [123]:
def ConsultaSQL(sql, conf):
    conn = psycopg2.connect(conf)
    try:
        cursor = conn.cursor()
        cursor.execute(sql)
        colnames = []
        colnames = [desc[0] for desc in cursor.description]
        linhas = cursor.fetchall()
        conn.commit()
        cursor.close()
        conn.close()

    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    df = pd.DataFrame(data = linhas, columns=colnames)
    #df.columns = colnames
    #df.append(linhas) 
    return df

def ConsultarBanco(caminho, conf):
    with open(caminho, 'r') as arquivo:
        consulta_sql = arquivo.read()
        consulta_sql = re.sub(r'[\ufeff]', '', consulta_sql)
        nome_arquivo = os.path.splitext(os.path.basename(caminho))[0]
        print(nome_arquivo)
    return(ConsultaSQL(consulta_sql, conf))

### Ler base

In [124]:
df = pd.read_parquet('./Base/base2010a2023corrigida_filtrada_materialidade 2025.03.01.parquet')

In [125]:
df

,ano_exercicio,ds_municipio,ds_orgao,cod_subelemento,vl_despesa,cd_municipio_ibge
0,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0100,"6,073,317.27","3,500,105.00"
1,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0300,"2,313,660.01","3,500,105.00"
2,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0400,"592,488.71","3,500,105.00"
3,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1101,"19,430,054.94","3,500,105.00"
4,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1137,859.22,"3,500,105.00"
...,...,...,...,...,...,...
474427,2023,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,5199,"424,122.54","3,533,809.00"
474428,2023,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,5200,"957,348.57","3,533,809.00"
474429,2023,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,9100,"223,698.24","3,533,809.00"
474430,2023,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,9300,"69,756.07","3,533,809.00"


In [126]:
df.cd_municipio_ibge = df.cd_municipio_ibge.astype(int)

In [127]:
df.value_counts('ano_exercicio')

ano_exercicio
2023    38483
2022    38334
2021    36736
2020    35983
2019    34476
2018    33747
2017    33310
2014    32927
2015    32875
2016    32611
2013    32535
2012    30988
2011    30945
2010    30482
Name: count, dtype: int64

### Incluir População
https://repositorio.seade.gov.br/dataset/populacao-residente-estado-de-sao-paulo/resource/54a6f188-ee81-4dc1-bbc4-07353fe06b04

In [128]:
pop = pd.read_csv('./Base/serie_populacao2000a2023.csv', sep=';', encoding='ansi')
pop.rename(columns={'ano':'ano_exercicio', 'cod_ibge':'cd_municipio_ibge'}, inplace=True)

In [129]:
pop[['cd_municipio_ibge', 'ano_exercicio', 'pop_total']]

,cd_municipio_ibge,ano_exercicio,pop_total
0,"3,500,105.00","2,000.00","33,484.00"
1,"3,500,105.00","2,001.00","33,577.00"
2,"3,500,105.00","2,002.00","33,636.00"
3,"3,500,105.00","2,003.00","33,677.00"
4,"3,500,105.00","2,004.00","33,715.00"
...,...,...,...
15476,"3,557,303.00","2,020.00","11,116.00"
15477,"3,557,303.00","2,021.00","11,200.00"
15478,"3,557,303.00","2,022.00","11,286.00"
15479,"3,557,303.00","2,023.00","11,359.00"


In [130]:
len(df.index) == len(df.merge(pop[['cd_municipio_ibge', 'ano_exercicio', 'pop_total']], how='left'))

True

In [131]:
df = df.merge(pop[['cd_municipio_ibge', 'ano_exercicio', 'pop_total']], how='left')

In [132]:
df.isna().any()

ano_exercicio        False
ds_municipio         False
ds_orgao             False
cod_subelemento      False
vl_despesa           False
cd_municipio_ibge    False
pop_total            False
dtype: bool

In [133]:
df.dtypes

ano_exercicio          int64
ds_municipio          object
ds_orgao              object
cod_subelemento       object
vl_despesa           float64
cd_municipio_ibge      int32
pop_total            float64
dtype: object

In [134]:
df.sample(5)

,ano_exercicio,ds_municipio,ds_orgao,cod_subelemento,vl_despesa,cd_municipio_ibge,pop_total
385050,2021,Piquerobi,PREFEITURA MUNICIPAL DE PIQUEROBI,3990,"142,923.97",3538303,"3,294.00"
82193,2012,Pirapora do Bom Jesus,PREFEITURA MUNICIPAL DE PIRAPORA DO BOM JESUS,4300,"86,112.86",3539103,"16,082.00"
317115,2019,Salesópolis,PREFEITURA MUNICIPAL DE SALESOPOLIS,3990,"51,848.28",3545001,"15,375.00"
462168,2023,Pompéia,PREFEITURA MUNICIPAL DE POMPÉIA,2100,"2,301,787.41",3540002,"20,154.00"
231549,2017,Elisiário,PREFEITURA MUNICIPAL DE ELISIÁRIO,7100,"603,026.83",3514924,"3,168.00"


### Incluir RCL

In [135]:
conf = 'dbname=audesp user=usr_leitura host=10.26.254.74'
caminho = './SQL/RCL.sql'
rcl = ConsultarBanco(caminho, conf)

RCL


OperationalError: connection to server at "10.26.254.74", port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
rcl.rename(columns={'ano':'ano_exercicio', 'cod_ibge':'cd_municipio_ibge'}, inplace=True)

In [ ]:
rcl.value_counts('ano_exercicio')

ano_exercicio
2010    644
2011    644
2012    644
2013    644
2014    644
2015    644
2016    644
2017    644
2018    644
2019    644
2020    644
2021    644
2022    644
2023    644
Name: count, dtype: int64

In [ ]:
len(df.index) ==len(df.merge(rcl, how='left'))

True

In [ ]:
df = df.merge(rcl, how='left')

In [ ]:
df.sample(5)

,ano_exercicio,ds_municipio,ds_orgao,cod_subelemento,vl_despesa,cd_municipio_ibge,pop_total,RCL
412217,2022,Itaoca,PREFEITURA MUNICIPAL DE ITAOCA,3200,"216,564.14",3522158,"3,421.00",24314263.24
294879,2019,Buritama,PREFEITURA MUNICIPAL DE BURITAMA,5199,"2,358,283.47",3508108,"16,932.00",66918653.28
75370,2012,Junqueirópolis,PREFEITURA MUNICIPAL DE JUNQUEIRÓPOLIS,3988,"47,426.47",3526001,"19,036.00",40204421.33
340307,2020,Jaguariúna,PREFEITURA MUNICIPAL DE JAGUARIÚNA,3923,"4,691.70",3524709,"57,043.00",392472470.05
371173,2021,Garça,PREFEITURA MUNICIPAL DE GARÇA,0400,"5,288,499.96",3516705,"42,382.00",166362539.53


### Incluir dados atemporais

In [ ]:
atemporais = pd.read_excel('./Base/informacoes_municipios.xlsx')

In [ ]:
atemporais.head()

,cod_ibge,Município,pib_nominal,atividade_principal,quantidade_total_vagas,vereadores,area,alunos_escola
0,3500105,adamantina,1054541,Demais serviços,2002,9,411,2359
1,3500204,adolfo,99734,"Agricultura, inclusive apoio à agricultura e a...",458,9,211,478
2,3500303,aguaí,942708,Demais serviços,1471,13,474,3489
3,3500709,agudos,2063021,Indústrias de transformação,2573,13,966,4062
4,3500758,alambari,100566,Demais serviços,409,9,159,683


In [ ]:
atemporais = atemporais.rename(columns={'cod_ibge':'cd_municipio_ibge'})

In [ ]:
len(df.index)

474432

In [ ]:
df.head()

,ano_exercicio,ds_municipio,ds_orgao,cod_subelemento,vl_despesa,cd_municipio_ibge,pop_total,RCL
0,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0100,"6,073,317.27",3500105,"33,794.00",69138047.59
1,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0300,"2,313,660.01",3500105,"33,794.00",69138047.59
2,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0400,"592,488.71",3500105,"33,794.00",69138047.59
3,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1101,"19,430,054.94",3500105,"33,794.00",69138047.59
4,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1137,859.22,3500105,"33,794.00",69138047.59


In [ ]:
atemporais

,cd_municipio_ibge,Município,pib_nominal,atividade_principal,quantidade_total_vagas,vereadores,area,alunos_escola
0,3500105,adamantina,1054541,Demais serviços,2002,9,411,2359
1,3500204,adolfo,99734,"Agricultura, inclusive apoio à agricultura e a...",458,9,211,478
2,3500303,aguaí,942708,Demais serviços,1471,13,474,3489
3,3500709,agudos,2063021,Indústrias de transformação,2573,13,966,4062
4,3500758,alambari,100566,Demais serviços,409,9,159,683
...,...,...,...,...,...,...,...,...
639,3500600,águas de são pedro,125145,Demais serviços,510,9,3,598
640,3501202,álvares florence,105910,"Agricultura, inclusive apoio à agricultura e a...",509,9,362,381
641,3501301,álvares machado,509445,Demais serviços,917,9,347,3081
642,3501400,álvaro de carvalho,49483,"Administração, defesa, educação e saúde públic...",245,9,153,371


In [ ]:
len(df.index)==len(df.merge(atemporais, how='left').index)

True

In [ ]:
df.merge(atemporais, how='left').isna().any()

ano_exercicio             False
ds_municipio              False
ds_orgao                  False
cod_subelemento           False
vl_despesa                False
cd_municipio_ibge         False
pop_total                 False
RCL                       False
Município                 False
pib_nominal               False
atividade_principal       False
quantidade_total_vagas    False
vereadores                False
area                      False
alunos_escola             False
dtype: bool

In [ ]:
df = df.merge(atemporais, how='left')

In [ ]:
df.sample(5)

,ano_exercicio,ds_municipio,ds_orgao,cod_subelemento,vl_despesa,cd_municipio_ibge,pop_total,RCL,Município,pib_nominal,atividade_principal,quantidade_total_vagas,vereadores,area,alunos_escola
55903,2011,Santo Antônio do Jardim,PREFEITURA MUNICIPAL DE SANTO ANTONIO DO JARDIM,3039,"750,610.64",3548104,"5,961.00",11617772.49,santo antônio do jardim,137159,Demais serviços,391,9,109,557
289362,2018,Valparaíso,PREFEITURA MUNICIPAL DE VALPARAÍSO,3910,"36,168.54",3556305,"23,903.00",82852966.74,valparaíso,989934,Indústrias de transformação,1164,11,857,2540
391387,2021,Sarapuí,PREFEITURA MUNICIPAL DE SARAPUÍ,3917,"12,088.27",3551108,"10,276.00",38765594.43,sarapuí,150274,Demais serviços,588,9,352,1241
305079,2019,Jacareí,PREFEITURA MUNICIPAL DE JACAREÍ,9200,"3,481,538.92",3524402,"234,877.00",822353396.39,jacareí,11696354,Indústrias de transformação,8382,13,464,18245
462164,2023,Pompéia,PREFEITURA MUNICIPAL DE POMPÉIA,1199,"2,837,251.46",3540002,"20,154.00",145090372.83,pompéia,825898,Demais serviços,1673,11,784,1715


### Incluir PIB

In [ ]:
pib = pd.read_parquet('./Base/Pib Municipal forecast 2022 e 2023.parquet')

In [ ]:
pib = pib.rename(columns={'Exercício':'ano_exercicio', 'cod_municipio_ibge':'cd_municipio_ibge'}).drop('Municípios', axis=1)

In [ ]:
df.head()

,ano_exercicio,ds_municipio,ds_orgao,cod_subelemento,vl_despesa,cd_municipio_ibge,pop_total,RCL,Município,pib_nominal,atividade_principal,quantidade_total_vagas,vereadores,area,alunos_escola
0,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0100,"6,073,317.27",3500105,"33,794.00",69138047.59,adamantina,1054541,Demais serviços,2002,9,411,2359
1,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0300,"2,313,660.01",3500105,"33,794.00",69138047.59,adamantina,1054541,Demais serviços,2002,9,411,2359
2,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0400,"592,488.71",3500105,"33,794.00",69138047.59,adamantina,1054541,Demais serviços,2002,9,411,2359
3,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1101,"19,430,054.94",3500105,"33,794.00",69138047.59,adamantina,1054541,Demais serviços,2002,9,411,2359
4,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1137,859.22,3500105,"33,794.00",69138047.59,adamantina,1054541,Demais serviços,2002,9,411,2359


In [ ]:
pib.head()

,ano_exercicio,cd_municipio_ibge,Agropecuária,Indústria,Administração Pública,Total (exclusive Administração Pública),Total geral,Impostos,PIB (1),PIB per Capita (2)
0,2021,3500105,"54,224.14","178,323.41","228,217.17","785,582.03","1,246,346.76","124,519.17","1,370,865.93","40,475.54"
1,2021,3500204,"22,826.15","17,782.94","27,140.21","42,622.33","110,371.62","5,118.37","115,489.99","33,611.75"
2,2021,3500303,"121,795.92","358,840.73","151,739.51","437,984.53","1,070,360.69","244,376.57","1,314,737.26","36,637.52"
3,2021,3500402,"25,096.38","25,550.80","36,922.72","106,278.69","193,848.59","15,155.41","209,003.99","26,771.36"
4,2021,3500501,"10,298.97","41,438.68","114,950.72","372,529.72","539,218.08","46,385.86","585,603.94","31,760.71"


In [ ]:
df.merge(pib, how='left')

,ano_exercicio,ds_municipio,ds_orgao,cod_subelemento,vl_despesa,cd_municipio_ibge,pop_total,RCL,Município,pib_nominal,...,area,alunos_escola,Agropecuária,Indústria,Administração Pública,Total (exclusive Administração Pública),Total geral,Impostos,PIB (1),PIB per Capita (2)
0,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0100,"6,073,317.27",3500105,"33,794.00",69138047.59,adamantina,1054541,...,411,2359,"33,606.84","113,294.52","92,451.64","346,461.65","585,814.65","53,275.85","639,090.51","18,911.36"
1,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0300,"2,313,660.01",3500105,"33,794.00",69138047.59,adamantina,1054541,...,411,2359,"33,606.84","113,294.52","92,451.64","346,461.65","585,814.65","53,275.85","639,090.51","18,911.36"
2,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0400,"592,488.71",3500105,"33,794.00",69138047.59,adamantina,1054541,...,411,2359,"33,606.84","113,294.52","92,451.64","346,461.65","585,814.65","53,275.85","639,090.51","18,911.36"
3,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1101,"19,430,054.94",3500105,"33,794.00",69138047.59,adamantina,1054541,...,411,2359,"33,606.84","113,294.52","92,451.64","346,461.65","585,814.65","53,275.85","639,090.51","18,911.36"
4,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1137,859.22,3500105,"33,794.00",69138047.59,adamantina,1054541,...,411,2359,"33,606.84","113,294.52","92,451.64","346,461.65","585,814.65","53,275.85","639,090.51","18,911.36"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474427,2023,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,5199,"424,122.54",3533809,"2,490.00",24890330.62,óleo,88221,...,198,294,"89,447.04","6,434.24","19,302.47","24,310.45","139,494.21","2,943.62","142,437.83","56,498.91"
474428,2023,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,5200,"957,348.57",3533809,"2,490.00",24890330.62,óleo,88221,...,198,294,"89,447.04","6,434.24","19,302.47","24,310.45","139,494.21","2,943.62","142,437.83","56,498.91"
474429,2023,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,9100,"223,698.24",3533809,"2,490.00",24890330.62,óleo,88221,...,198,294,"89,447.04","6,434.24","19,302.47","24,310.45","139,494.21","2,943.62","142,437.83","56,498.91"
474430,2023,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,9300,"69,756.07",3533809,"2,490.00",24890330.62,óleo,88221,...,198,294,"89,447.04","6,434.24","19,302.47","24,310.45","139,494.21","2,943.62","142,437.83","56,498.91"


In [ ]:
len(df.index)==len(df.merge(pib, how='left').index)

True

In [ ]:
df = df.merge(pib, how='left')

### Incluir Quadro de pessoal

In [ ]:
quadro = pd.read_parquet('./Base/quadro_pessoal.parquet')

In [ ]:
quadro.head()

Exercício de Atividade,cd_municipio_ibge,ano_exercicio,Efetivo,Efetivo em Comissão,Exclusivamente em Comissão
0,3500105,2016,929.00,NaN,47.00
1,3500105,2017,939.00,NaN,59.00
2,3500105,2018,964.00,NaN,52.00
3,3500105,2019,"1,050.00",NaN,31.00
4,3500105,2020,"1,064.00",NaN,33.00


In [ ]:
len(df.index)==len(df.merge(quadro, how='left'))

True

In [ ]:
df = df.merge(quadro, how='left')

### Incluir dados do censo escolar

In [140]:
censo = pd.read_parquet('./Base/censo2010_2023.parquet')

In [137]:
censo.head()

,ano_exercicio,cd_municipio_ibge,QT_MAT_BAS,QT_MAT_INF,QT_MAT_FUND,QT_DOC_BAS,QT_DOC_INF,QT_DOC_FUND
0,2010,3500105,"2,559.00",947.00,"1,564.00",157.00,62.00,94.00
1,2010,3500204,530.00,33.00,497.00,34.00,2.00,32.00
2,2010,3500303,"4,190.00",885.00,"2,693.00",203.00,43.00,139.00
3,2010,3500402,629.00,164.00,450.00,41.00,11.00,30.00
4,2010,3500501,"2,388.00",720.00,"1,572.00",141.00,47.00,88.00


In [141]:
censo.loc[censo.ano_exercicio ==2020]

,ano_exercicio,cd_municipio_ibge,QT_MAT_BAS,QT_MAT_INF,QT_MAT_FUND,QT_DOC_BAS,QT_DOC_INF,QT_DOC_FUND
6450,2020,3500105,"2,380.00","1,086.00","1,272.00",164.00,74.00,89.00
6451,2020,3500204,611.00,115.00,496.00,49.00,9.00,40.00
6452,2020,3500303,"4,010.00","1,283.00","2,551.00",230.00,77.00,150.00
6453,2020,3500402,642.00,267.00,375.00,53.00,31.00,27.00
6454,2020,3500501,"2,331.00",794.00,"1,458.00",200.00,93.00,107.00
...,...,...,...,...,...,...,...,...
7090,2020,3557006,"12,511.00","5,476.00","6,997.00",490.00,207.00,298.00
7091,2020,3557105,"8,020.00","3,675.00","4,326.00",443.00,187.00,255.00
7092,2020,3557154,294.00,113.00,181.00,18.00,6.00,12.00
7093,2020,3557204,"1,451.00",513.00,938.00,125.00,51.00,77.00


In [144]:
censo.columns

Index(['ano_exercicio', 'cd_municipio_ibge', 'QT_MAT_BAS', 'QT_MAT_INF',
       'QT_MAT_FUND', 'QT_DOC_BAS', 'QT_DOC_INF', 'QT_DOC_FUND'],
      dtype='object')

In [151]:
len(df.merge(censo, how='left'))==len(df.index)

True

In [ ]:
df = df.merge(censo, how='left')

### Incluir Receitas

In [157]:
receitas = pd.read_parquet('./Base/Receitas_2010a2023.parquet')

In [161]:
receitas.columns

Index(['ano_exercicio', 'receitas_correntes', 'receitas_capital',
       'receita_total', 'cd_municipio_ibge'],
      dtype='object')

In [159]:
len(df.index)==len(df.merge(receitas, how='left'))

True

In [160]:
df = df.merge(receitas, how='left')

### Incluir leitos hospitalares

In [168]:
df = pd.read_parquet('./Base/base_pronta_2023 2025.03.03.parquet')

In [169]:
leitos = pd.read_excel('./Base/saude_leitos_mun_ano.xlsx')

In [177]:
leitos.columns

Index(['cod_ibge', 'localidades', 'periodos', 'leitos_total', 'leitos_hab',
       'leitos sus', 'leitos_sus_hab', 'leitos não sus', 'leitos_naosus_hab'],
      dtype='object')

In [174]:
len(df.index)==len(df.merge(leitos.rename(columns={'cod_ibge':'cd_municipio_ibge', 
                                                   'periodos':'ano_exercicio'}).drop(columns='localidades'), how='left'))

True

In [175]:
df = df.merge(leitos.rename(columns={'cod_ibge':'cd_municipio_ibge', 
                                     'periodos':'ano_exercicio'}).drop(columns='localidades'), how='left')

In [182]:
colunas = ['leitos_total', 'leitos_hab', 'leitos sus', 'leitos_sus_hab', 'leitos não sus', 'leitos_naosus_hab']
df[colunas] = df[colunas].fillna(0)

### Salvar base pronta

In [178]:
cols = ['ano_exercicio', 'cd_municipio_ibge', 'ds_municipio', 'cod_subelemento',
       'vl_despesa', 'pop_total', 'RCL', 'pib_nominal', 'atividade_principal', 
       'quantidade_total_vagas', 'vereadores', 'area', 'alunos_escola', 'Agropecuária', 
       'Indústria', 'Administração Pública', 'Total  (exclusive Administração Pública)',
       'Total geral', 'Impostos', 'PIB (1)', 'PIB per Capita (2)', 'Efetivo',
       'Efetivo em Comissão', 'Exclusivamente em Comissão', 'QT_MAT_BAS',
       'QT_MAT_INF', 'QT_MAT_FUND', 'QT_DOC_BAS', 'QT_DOC_INF', 'QT_DOC_FUND', 
       'receitas_correntes', 'receitas_capital', 'receita_total',
       'leitos_total', 'leitos_hab', 'leitos sus', 'leitos_sus_hab', 'leitos não sus', 'leitos_naosus_hab']

In [183]:
df[cols].isna().any()

ano_exercicio                               False
cd_municipio_ibge                           False
ds_municipio                                False
cod_subelemento                             False
vl_despesa                                  False
pop_total                                   False
RCL                                         False
pib_nominal                                 False
atividade_principal                         False
quantidade_total_vagas                      False
vereadores                                  False
area                                        False
alunos_escola                               False
Agropecuária                                False
Indústria                                   False
Administração Pública                       False
Total  (exclusive Administração Pública)    False
Total geral                                 False
Impostos                                    False
PIB (1)                                     False


In [184]:
from datetime import datetime

# Obter a data atual no formato AAAA.MM.DD
hoje = datetime.today().strftime('%Y.%m.%d')

In [185]:
df[cols].to_parquet(f'./Base/base_pronta_2023 {hoje}.parquet', index=False)